In [2]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#import rasterio
import re
import rtree
import shapely
import pickle
import data_eng.az_proc as ap
#from cartopy import crs
import collections
import cv2
import math
from glob import glob

# Confirm Tiles Are Downloads (Or Download Tiles)

In [2]:
sub_directories
img_annotation_path 

#Identify the tiles that the images come from
tile_id = []
for image in os.listdir(os.path.join(parent,annotation_directory,anno)): #iterate through the images
    tile_id.append(image.rsplit("_",1)[0])
tile_id = np.unique(np.array(tile_id))
#print(tile_id)

#identify the tiles that are already downloaded
downloaded_tiles = []
for tiles in os.listdir(os.path.join(parent,annotation_directory,"tiles")): #iterate through the images
    downloaded_tiles.append(tiles.split(".",1)[0])
downloaded_tiles= np.unique(np.array([downloaded_tiles]))
#print(downloaded_tiles)

tiles_labeled = "tile_name_tile_url_labeled.npy"
np_tiles_labeled = np.load(tiles_labeled)
for labeled_image_tile_id in tile_id: # the tiles from images 
    if labeled_image_tile_id not in tiles: #the downloaded times 
        #print(labeled_image_tile_id)
        #download the tiles if they are not in the tiles folder 
        for tile_name_url in np_tiles_labeled[:,]:
            if labeled_image_tile_id == tile_name_url[0].split(".",1)[0]:
                ###Download tile
                ap.download_url(tile_name_url[1], os.path.join(parent,annotation_directory,"tiles"),
                                                         progress_updater=ap.DownloadProgressBar())


NameError: name 'sub_directories' is not defined

In [4]:
parent_directory = "C:\chips_allocation"
annotation_directory = "Check"
number_of_tiles =10
tiles_remaining = "tile_name_tile_url_remaining_expanded.npy"
tiles_labeled = "tile_name_tile_url_labeled.npy"

In [5]:
    # Index through each image in a tile
    # Calculate the pixel location of each image within the tile
        # The Save the NW SE pixel coordinates of the Image
        # Save the Centroid of the Image 


Unlabeled Tiles (801, 2)


In [51]:
#Gather the locations of tiles that have already been downlaoded and verified 
path_verification_set1 = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verification_set1/unverified_images/student_reviewed_unverified_images"
tifs_verification_set1 = glob(path_verification_set1 + "/**/*.tif", recursive = True)
print(len(tifs_verification_set1))

path_verification_set2 = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verification_set2/unverified_images/student_reviewed_unverified_images"
tifs_verification_set2 = glob(path_verification_set2 + "/**/*.tif", recursive = True)
print(len(tifs_verification_set2))

path_verification_set3 = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verification_set3/unverified_images/student_reviewed_unverified_images"  
tifs_verification_set3 = glob(path_verification_set3 + "/**/*.tif", recursive = True)
print(len(tifs_verification_set3))

#identify tiles that hve been downloaded 
downloaded_tiles_in_verification_sets = tifs_verification_set1 + tifs_verification_set2 + tifs_verification_set3
downloaded_tiles_in_verification_sets

98
542
493


['//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verification_set1/unverified_images/student_reviewed_unverified_images\\Jaewon\\student_reviewed_images5\\tiles\\co_60cm_2019_40103_m_4010318_sw_13_060_20190913.tif',
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verification_set1/unverified_images/student_reviewed_unverified_images\\Jaewon\\student_reviewed_images5\\tiles\\co_60cm_2019_40104_m_4010423_ne_13_060_20190914.tif',
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verification_set1/unverified_images/student_reviewed_unverified_images\\Jaewon\\student_reviewed_images6\\tiles\\pa_60cm_2019_41078_m_4107803_se_17_060_20190917.tif',
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verification_set1/unverified_images/student_reviewed_unverified_images\\Jaewon\\student_reviewed_images6\\tiles\\pa_60cm_2019_41078_m_4107817_nw_17_060_20190927.tif',
 '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verification_set1/unverified_images/student_reviewed_unverifie

In [52]:
#Load file that tracks the images that have been labeled 
tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)

In [53]:
#create folder to hold tiles in completed dataset
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"
path_to_complete_dataset = "verified/complete_dataset"
path_to_tiles_folder_complete_dataset = os.path.join(parent_directory, path_to_complete_dataset,"tiles")
os.makedirs(path_to_tiles_folder_complete_dataset, exist_ok=True)

In [102]:
#Move already downloaded tiles to completed dataset
for path in downloaded_tiles_in_verification_sets:
    base = os.path.basename(path)
    tif = os.path.splitext(base)[0] #name of tif with the extension removed
    #if tif in tile_img_annotation[:,0]:
    #    shutil.move(path, os.path.join(path_to_tiles_folder_complete_dataset,base)) # copy images with matching .xml files in the "chips_tank" folder

In [108]:
#Make a list of the tiles moved to completed dataset
tiles_downloaded = os.listdir(path_to_tiles_folder_complete_dataset)
tiles_downloaded_temp = []
for tile in tiles_downloaded:
     tiles_downloaded_temp.append(os.path.splitext(tile)[0]) #name of tif with the extension removed
tiles_downloaded = np.array(tiles_downloaded_temp)

## Download remaining tiles that correspond to ONLY to verified images

In [70]:
#Gather the locations of tiles that have already been downlaoded and verified 
path_positive_images_complete_dataset = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verified/complete_dataset/chips_positive"
jpg_path_positive_images_complete_dataset = glob(path_positive_images_complete_dataset + "/*.jpg", recursive = True)
print(len(jpg_path_positive_images_complete_dataset))

tiles_to_be_downloaded = []
for path in jpg_path_positive_images_complete_dataset:
    base = os.path.basename(path)
    img = os.path.splitext(base)[0] #name of tif with the extension removed
    tiles_to_be_downloaded.append(img.rsplit("_",1)[0])
tiles_to_be_downloaded = np.array(tiles_to_be_downloaded)

tiles_to_be_downloaded = np.unique(tiles_to_be_downloaded)

13362


In [203]:
tiles_to_be_downloaded.size

1073

In [153]:
#Identify tiles that have not already been downloaded
tile_names_to_download = []
for tile in tiles_to_be_downloaded: #index over the downloaded tiled
    if tile not in tiles_downloaded: #index over the tiles that should be downloded
        tile_names_to_download.append(tile)

In [166]:
tile_names_to_download[0].split("_",4)
tile_names_to_download[0].split("_",4)

['al', '60cm', '2019', '30088', 'm_3008816_se_16_060_20191115']

In [ ]:
#FIX double issue

In [207]:
tiles_labeled = "tile_name_tile_url_labeled.npy"
np_tiles_labeled = np.load(tiles_labeled)
np_tiles_labeled

blob_root = 'https://naipblobs.blob.core.windows.net/naip'
#blob_root = 'https://naipeuwest.blob.core.windows.net/naip'  
tile_names = []
tile_urls = []

for tile in tile_names_to_download:
    tile_name = tile.split("_",4)[4] #+ ".tif"
    
    tile_name = [string for string in np_tiles_labeled[:,0] if tile_name in string]  #check if the tile_name is in the string contained in the array

['m_3208433_nw_16_060_20191003.tif', 'm_3208433_nw_16_060_20191003.tif']

In [206]:
#Check how many border tiles there are 
counter = 0
for tile in tiles_to_be_downloaded:
    tile_name = tile.split("_",4)[4] #+ ".tif"
    tile_name = [string for string in np_tiles_labeled[:,0] if tile_name in string]  #check if the tile_name is in the string contained in the array
    if len(tile_name) > 1:
        print(tile_name)
        counter += 1
counter

['m_3208433_nw_16_060_20191003.tif', 'm_3208433_nw_16_060_20191003.tif']
['m_3411956_nw_11_060_20180724_20190209.tif', 'm_3411956_nw_11_060_20180724_20190209.tif']
['m_3311822_ne_11_060_20180723_20190209.tif', 'm_3311822_ne_11_060_20180723_20190209.tif']
['m_4207260_se_18_060_20180823.tif', 'm_4207260_se_18_060_20180823.tif']
['m_4109029_sw_15_060_20190802.tif', 'm_4109029_sw_15_060_20190802.tif']
['m_4109035_ne_15_060_20190801.tif', 'm_4109035_ne_15_060_20190801.tif']
['m_4209027_se_15_060_20190801.tif', 'm_4209027_se_15_060_20190801.tif']
['m_4109029_sw_15_060_20190802.tif', 'm_4109029_sw_15_060_20190802.tif']
['m_4109035_ne_15_060_20190801.tif', 'm_4109035_ne_15_060_20190801.tif']
['m_4209027_se_15_060_20190801.tif', 'm_4209027_se_15_060_20190801.tif']
['m_3708701_se_16_060_20180905_20190221.tif', 'm_3708701_se_16_060_20180905.tif']
['m_3708701_sw_16_060_20180905_20181211.tif', 'm_3708701_sw_16_060_20180905.tif']
['m_3708704_nw_16_060_20180725.tif', 'm_3708704_nw_16_060_20180725_201

36

In [201]:
np_tiles_labeled[np_tiles_labeled[:,0] == 'm_4108616_sw_16_060_20180707_20181211.tif']

array([['m_4108616_sw_16_060_20180707_20181211.tif',
        'https://naipblobs.blob.core.windows.net/naip/v002/mi/2018/mi_060cm_2018/41086/m_4108616_sw_16_060_20180707_20181211.tif'],
       ['m_4108616_sw_16_060_20180707_20181211.tif',
        'https://naipblobs.blob.core.windows.net/naip/v002/in/2018/in_060cm_2018/41086/m_4108616_sw_16_060_20180707_20181211.tif']],
      dtype='<U120')

In [198]:
#https://www.kite.com/python/answers/how-to-check-if-a-list-contains-a-substring-in-python
https://naipblobs.blob.core.windows.net/naip/v002/in/2018/in_060cm_2018/41086/m_4108616_sw_16_060_20180707_20181211.tif

['m_3908010_sw_17_060_20180707.tif']

In [189]:
np_tiles_labeled[:,0][0]

'm_4207140_se_19_060_20181016.tif'

In [176]:
tile_urls[0]



'https://naipblobs.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/32115/m_3211520_se_11_060_20190305.tif'

In [180]:
import pandas as pd
pd.DataFrame(np.load("image_download_azure/tile_name_tile_url_complete_array.npy")).to_csv("image_download_azure/tile_name_tile_url_complete_array.csv")

In [179]:
x

array([['m_2408126_ne_17_060_20191128.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/fl/2019/fl_60cm_2019/24081/m_2408126_ne_17_060_20191128.tif'],
       ['m_2408126_se_17_060_20191128.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/fl/2019/fl_60cm_2019/24081/m_2408126_se_17_060_20191128.tif'],
       ['m_2508014_se_17_060_20191126.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/fl/2019/fl_60cm_2019/25080/m_2508014_se_17_060_20191126.tif'],
       ...,
       ['m_4812237_ne_10_060_20190806.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/wa/2019/wa_60cm_2019/48122/m_4812237_ne_10_060_20190806.tif'],
       ['m_4812258_se_10_060_20191011.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/wa/2019/wa_60cm_2019/48122/m_4812258_se_10_060_20191011.tif'],
       ['m_4812258_sw_10_060_20191029.tif',
        'https://naipeuwest.blob.core.windows.net/naip/v002/wa/2019/wa_60cm_2019/48122/m_4812258_sw_10_060_20

In [127]:

#for tile in tiles_to_download:
#    print(tile+".tif")
    
    #print(np_tiles_labeled[np_tiles_labeled[:,0] == tile+".tif"])
            'https://naipblobs.blob.core.windows.net/naip/v002/va/2018/va_060cm_2018/36076/m_3607614_ne_18_060_20181001.tif'],

array([['m_4207140_se_19_060_20181016.tif',
        'https://naipblobs.blob.core.windows.net/naip/v002/ma/2018/ma_060cm_2018/42071/m_4207140_se_19_060_20181016.tif'],
       ['m_4207140_sw_19_060_20181016.tif',
        'https://naipblobs.blob.core.windows.net/naip/v002/ma/2018/ma_060cm_2018/42071/m_4207140_sw_19_060_20181016.tif'],
       ['m_3607614_ne_18_060_20181001.tif',
        'https://naipblobs.blob.core.windows.net/naip/v002/va/2018/va_060cm_2018/36076/m_3607614_ne_18_060_20181001.tif'],
       ...,
       ['m_3310236_ne_13_060_20180928.tif',
        'https://naipblobs.blob.core.windows.net/naip/v002/tx/2018/tx_060cm_2018/33102/m_3310236_ne_13_060_20180928.tif'],
       ['m_2909601_ne_14_060_20181116_20190426.tif',
        'https://naipblobs.blob.core.windows.net/naip/v002/tx/2018/tx_060cm_2018/29096/m_2909601_ne_14_060_20181116_20190426.tif'],
       ['m_2909521_se_15_060_20181204_20190503.tif',
        'https://naipblobs.blob.core.windows.net/naip/v002/tx/2018/tx_060cm_2018/2

In [144]:
np.where(np_tiles_labeled[:,0] == "m_3211520_se_11_060_20190305.tif")

(array([], dtype=int64),)

In [117]:
        #if not os.path.exists(os.path.join(path_to_tiles_folder_complete_dataset,base)): #skip over tiles that already exist in complete dataset

In [116]:
#remove tiles / images with no annottions

509

In [ ]:
#Download Tiles 

for labeled_image_tile_id in tile_id: # the tiles from images 
    if labeled_image_tile_id not in tiles: #the downloaded times 
        #print(labeled_image_tile_id)
        #download the tiles if they are not in the tiles folder 
        for tile_name_url in np_tiles_labeled[:,]:
            if labeled_image_tile_id == tile_name_url[0].split(".",1)[0]:
                ###Download tile
                ap.download_url(tile_name_url[1], os.path.join(parent,annotation_directory,"tiles"),
                                                         progress_updater=ap.DownloadProgressBar())

In [21]:

#downloaded_tiles_in_verification_sets = tifs_verification_set1 + tifs_verification_set2 + tifs_verification_set3